# Funkcje generacji klucza (ang. [*Key Derivation Function*](https://en.wikipedia.org/wiki/Key_derivation_function)) (KDF)

Składnia funkcji jest nastepująca:

$key = KDF( password, salt, cost)$

Są to fukcje jednokierukowe których koszt obliczenia wprzód jest dość znaczny. Parametr kosztu $cost$ dobiera się tak aby obliczenie zajmowało ok. 0.1ms. Parametr ten jest zazwyczaj niezmienny dla określonej implementacji. Przy takich nastawach koszt weryfikacji właściwego hasła jest zaniedbywalny, jednak przeprowadzenie ataku słownikowego na parametr $password$ jest bardzo złożone obliczeniowo. Parametr $salt$ jest liczbą pseudolosową, co uniemożliwia wykonanie obliczeń na zapas ze względu na rozmiar wymaganej bazy. Zazwyczaj wartość $salt$ jest zapisana w bazie systemowej razem z warościa $key$.

Technikę tę nazywa się soleniem hasła lub rozciąganiem klucza (ang. [*key streching*](https://en.wikipedia.org/wiki/Key_stretching)). Odmianę w której paramer $salt$ jest zapominany i do porównania należy wykonać przeszukiwania możliwych wartości tego parametru nazywa się wzmacnianiem klucza (ang. *key strengthening*).

## PBKDF2

In [1]:
import os, binascii
# Przykładowa generacja klucza dla AES256
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA256
salt = os.urandom(16)
password = b'password123'
key = PBKDF2(password, salt, 64, count=1000, hmac_hash_module=SHA256)
print("Salt:", binascii.hexlify(salt))
print("Derived key:", binascii.hexlify(key))

Salt: b'188ac80dd5895f7c750701d627794197'
Derived key: b'6515ede97ca92a7cdad867e71c221a562c858d564cc15e491bad311f1b7a63161a7bc31879cb2f0d03843927fcfeb4aa8cdc988e032069c7479a05fcf4af1a7e'


Zauważ, ze każde uruchomienie kodu powyżej daje inny wynik bo zmienia sie parametr $salt$.

## Bcrypt

In [2]:
import os, binascii
from base64 import b64encode
from Crypto.Protocol.KDF import bcrypt
from Crypto.Hash import SHA3_256
salt = os.urandom(16)
password = b'password123'
# Bcrypt domyslnie nie soli haseł, można to zrobić na zewnątrz algorytmu
b64pwd = b64encode(SHA3_256.new(salt+password).digest())
bcrypt_hash = bcrypt(b64pwd, 12)
print("Salt:", binascii.hexlify(salt))
print("Hashed password:", binascii.hexlify(bcrypt_hash))

Salt: b'06c80f66271fdfb26a4b6f34beec171e'
Hashed password: b'24326124313224574a4e575631787a67785954467a51784c50696a37654d4336645963537735715041332f48637a41566947306649694d41676e7a57'


Obiekt zawiera interfejs do sprawdzenia poprawności hasła wprowadzonego przez użytkownika.

In [3]:
from Crypto.Protocol.KDF import bcrypt_check
user_pass = b"password123"
try:
    b64pwd = b64encode(SHA3_256.new(salt+user_pass).digest())
    bcrypt_check(b64pwd, bcrypt_hash)
    print("OK")
except ValueError:
    print("Incorrect password")

OK


## [Scrypt](https://en.wikipedia.org/wiki/Scrypt)

Scrypt wewnętrznie używa funkcji PBKDF2. Doddakowe parametry służą lepszemu dopasowaniu kosztu obliczeń do architektury systemu (wg. mnie to bez sensu, bo nie da się przewidzieć co kryptoanalityk będzie miał do dyspozycji).

Składnia:

scrypt(password, salt, N, r, p, buflen).
    The parameters r, p, and buflen must satisfy r * p < 2^30 and
    buflen <= (2^32 - 1) * 32. The parameter N must be a power of 2
    greater than 1. N, r and p must all be positive.

In [4]:
import os,binascii
from Crypto.Protocol.KDF import scrypt
password = b'password123'
salt = os.urandom(16)
# Parametry dogodne dla sprawdzania haseł t<=100ms
key = scrypt(password, salt, 16, N=2**14, r=8, p=1) 
# Parametry dogodne dla generacji kluczy sesyjnych (np. szyfrowanie plików) t<=5s
# key = scrypt(password, salt, 16, N=2**20, r=8, p=1) 
print("Salt:", binascii.hexlify(salt))
print("Derived key:", binascii.hexlify(key))

Salt: b'192ee0e4a8316d396b3e79739e631300'
Derived key: b'17db61aa377fe5b318573b6eaa3e7e2d'


## Argon2

Obecnie (kwiecień 2021) uważa się, że funkcja Argon2 jest lepsza od innych funkcji KDF i powinna być używana gdy tylko jest to możliwe.

In [5]:
import os,binascii,argon2
salt = os.urandom(32)
password = b'password123'
hash = argon2.hash_password_raw(password=password,salt=salt,
    time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32, type=argon2.low_level.Type.ID)
print("Salt:", binascii.hexlify(salt))
print("Derived key:", binascii.hexlify(key))

Salt: b'19c74c0464037badb17a8e6c67b53c2420d5ff5cd020886d224eb3fa13dc9621'
Derived key: b'17db61aa377fe5b318573b6eaa3e7e2d'


Należy pamiętać, że tym razem w bazie należy przechowywać nie tylko sól ale wszystkie pozostałe parametry obliczeń. Biblioteka zapewnia gotowy prymityw służący do tego celu. W poniższym kodzie sól jest obliczana automatycznie i nie wymaga interwencji programisty.

In [6]:
import os,binascii,argon2
a2hasher = argon2.PasswordHasher(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32, salt_len=16)
password = b'password123'
phash = a2hasher.hash(password)
print("Argon2 hash (random salt):", phash)

def passwd_verify(hasher, phash, passwd) :
    try:
        hasher.verify(phash, passwd)
        return True
    except:
        return False

print("Is 'password123` a correct password:", passwd_verify(a2hasher, phash, b"password123"))
print("Is '123password` a correct password:", passwd_verify(a2hasher, phash, b"123password"))

Argon2 hash (random salt): $argon2id$v=19$m=32768,t=16,p=2$YYWNmdlhOK7RHhRyNJqE1w$3tfGVHnUoaBAh+c7KxfcsgD3cdkeqGXos2L9V9zi72Q
Is 'password123` a correct password: True
Is '123password` a correct password: False
